In [1]:
import ray.tune as tune
from ray.rllib.agents.ppo import PPOTrainer
from ray.tune import register_env
from envs.env_creator import env_creator, ibgym_env_creator_rllib
from envs.IBGym_mod_envs import IBGymModded
from ppo.policy import LSTMPPOPolicy

## Configure Training

In [2]:


register_env("IBGym-v1", ibgym_env_creator_rllib)

config = {
    # Environment (RLlib understands openAI gym registered strings).
    "env": "IBGym-v1",
    # Use 2 environment workers (aka "rollout workers") that parallelly
    # collect samples from their own environment clone(s).
    "num_workers": 12,
    "num_gpus": 1,

    # Change this to "framework: torch", if you are using PyTorch.
    # Also, use "framework: tf2" for tf2.x eager execution.
    "framework": "tf",
    "entropy_coeff": 0.0001,
    # "entropy_coeff_schedule":PiecewiseSchedule(endpoints=[(0, 0.01), (143000, 0.00001)]),
    "lr": 3e-5,
    "gamma": 0.994,
    "clip_param": 0.2,
    "seed": 5321,
    "num_sgd_iter": 2,
    "sgd_minibatch_size": 1000,

    # "vf_loss_coeff": 1e-9,
    # "vf_clip_param": 1e7,
    # Tweak the default model provided automatically by RLlib,
    # given the environment's observation- and action spaces.
    "model": {
        # == LSTM ==
        # Whether to wrap the model with an LSTM.
        "use_lstm": False,
        # Max seq len for training the LSTM, defaults to 20.
        #"max_seq_len": 30,
        # Size of the LSTM cell.
        #"lstm_cell_size": 64,
        # "use_attention": True,
        # "attention_num_transformer_units": 2,
        # "attention_dim": 128,
        # "vf_share_layers": True,
        # "fcnet_hiddens": [32, 32, 32],
        # "sgd_minibatch_size": 1024,
        "vf_share_layers": False,
        # Whether to feed a_{t-1} to LSTM (one-hot encoded if discrete).
        "lstm_use_prev_action": False,
        # Whether to feed r_{t-1} to LSTM.
        "lstm_use_prev_reward": False,
        # Whether the LSTM is time-major (TxBx..) or batch-major (BxTx..).
        "_time_major": False,
    },
    "train_batch_size": 32000,
    "timesteps_per_iteration": 32000,
    # "output": "tmp/ib-out",
    # Set up a separate evaluation worker set for the
    # `trainer.evaluate()` call after training (see below).
    "evaluation_num_workers": 3,
    # Only for evaluation runs, render the env.
    "evaluation_config": {
        "render_env": False,
    },
}

In [ ]:
results = tune.run(
        PPOTrainer,
        config=config,
        name="industrial_benchmark",
        local_dir="tmp/ray_exp_logs",
        checkpoint_freq=5,
        # stop={"training_iteration": 5},
        sync_config=tune.SyncConfig(
            syncer=None  # Disable syncing
        )
    )

2022-10-15 17:45:23,870	INFO services.py:1470 -- View the Ray dashboard at http://127.0.0.1:8265
(PPOTrainer pid=18350) 2022-10-15 17:45:29,741	INFO trainer.py:2332 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
(PPOTrainer pid=18350) 2022-10-15 17:45:29,745	WARNING trainer.py:2540 -- You have specified 3 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Trainer.train()`. Instead, you will have to call `Trainer.evaluate()` manually in order to trigger an evaluation run.
(PPOTrainer pid=18350) 2022-10-15 17:45:29,745	WARNING ppo.py:386 -- `train_batch_size` (32000) cannot be achieved with your other settings (num_workers=12 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment

Trial name,status,loc
PPOTrainer_IBGym-v1_62e25_00000,RUNNING,192.168.0.185:18350


Trial name,status,loc
PPOTrainer_IBGym-v1_62e25_00000,RUNNING,192.168.0.185:18350


Trial name,status,loc
PPOTrainer_IBGym-v1_62e25_00000,RUNNING,192.168.0.185:18350


Result for PPOTrainer_IBGym-v1_62e25_00000:
  agent_timesteps_total: 63984
  counters:
    num_agent_steps_sampled: 63984
    num_agent_steps_trained: 63984
    num_env_steps_sampled: 63984
    num_env_steps_trained: 63984
  custom_metrics: {}
  date: 2022-10-15_17-45-54
  done: false
  episode_len_mean: 1000.0
  episode_media: {}
  episode_reward_max: -242643.40926283607
  episode_reward_mean: -352599.23873893346
  episode_reward_min: -694923.1384058335
  episodes_this_iter: 60
  episodes_total: 60
  experiment_id: 416915bca0d6482fa48dbfee58d8681b
  hostname: hamza-Legion-5-15ACH6H
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 2.9999999242136255e-05
          entropy: 3.2951600551605225
          entropy_coeff: 9.999999747378752e-05
          kl: 0.0006728667649440467
          model: {}
          policy_loss: -0.012838481925427914
          total_loss: 9.986967086791992
          vf_explained_var: -5.67

...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_IBGym-v1_62e25_00000,RUNNING,192.168.0.185:18350,285,2708.46,18235440,-190893,-187256,-198538,1000


Result for PPOTrainer_IBGym-v1_62e25_00000:
  agent_timesteps_total: 18299424
  counters:
    num_agent_steps_sampled: 18299424
    num_agent_steps_trained: 18299424
    num_env_steps_sampled: 18299424
    num_env_steps_trained: 18299424
  custom_metrics: {}
  date: 2022-10-15_18-31-13
  done: false
  episode_len_mean: 1000.0
  episode_media: {}
  episode_reward_max: -187256.28434776116
  episode_reward_mean: -190740.00314073593
  episode_reward_min: -201771.0343365065
  episodes_this_iter: 60
  episodes_total: 18288
  experiment_id: 416915bca0d6482fa48dbfee58d8681b
  hostname: hamza-Legion-5-15ACH6H
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 2.9999999242136255e-05
          entropy: 3.0986595153808594
          entropy_coeff: 9.999999747378752e-05
          kl: 0.0005480574327521026
          model: {}
          policy_loss: -0.0015350134344771504
          total_loss: 9.99826431274414
          vf_ex

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_IBGym-v1_62e25_00000,RUNNING,192.168.0.185:18350,286,2718.44,18299424,-190740,-187256,-201771,1000


Result for PPOTrainer_IBGym-v1_62e25_00000:
  agent_timesteps_total: 18363408
  counters:
    num_agent_steps_sampled: 18363408
    num_agent_steps_trained: 18363408
    num_env_steps_sampled: 18363408
    num_env_steps_trained: 18363408
  custom_metrics: {}
  date: 2022-10-15_18-31-23
  done: false
  episode_len_mean: 1000.0
  episode_media: {}
  episode_reward_max: -186405.0224335903
  episode_reward_mean: -190814.40596034896
  episode_reward_min: -205528.119498394
  episodes_this_iter: 72
  episodes_total: 18360
  experiment_id: 416915bca0d6482fa48dbfee58d8681b
  hostname: hamza-Legion-5-15ACH6H
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 2.9999999242136255e-05
          entropy: 3.0971672534942627
          entropy_coeff: 9.999999747378752e-05
          kl: 0.0006344734574668109
          model: {}
          policy_loss: 0.005250617396086454
          total_loss: 10.00506591796875
          vf_expla

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_IBGym-v1_62e25_00000,RUNNING,192.168.0.185:18350,287,2728.1,18363408,-190814,-186405,-205528,1000
